In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer.primitives import Sampler
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import numpy as np

In [3]:
# Function to create the inverse Quantum Fourier Transform (QFT†)
def qft_dagger(num_qubits):
    """Construct the inverse Quantum Fourier Transform (QFT†) circuit."""
    qft_circuit = QuantumCircuit(num_qubits)
    for i in range(num_qubits // 2):
        qft_circuit.swap(i, num_qubits - i - 1)
    for i in range(num_qubits):
        for j in range(i):
            qft_circuit.cp(-np.pi / (2 ** (i - j)), j, i)
        qft_circuit.h(i)
    return qft_circuit


In [4]:

# Function to build the QPE circuit
def phase_estimation(unitary, eigenstate, num_ancillae):
    """Construct the Quantum Phase Estimation circuit."""
    num_unitary_qubits = unitary.num_qubits

    # Create Quantum and Classical Registers
    ancilla_reg = QuantumRegister(num_ancillae, name="ancilla")
    unitary_reg = QuantumRegister(num_unitary_qubits, name="unitary")
    classical_reg = ClassicalRegister(num_ancillae, name="classical")
    qc = QuantumCircuit(ancilla_reg, unitary_reg, classical_reg)

    # Initialize ancilla qubits in superposition
    qc.h(ancilla_reg)

    # Add eigenstate to the circuit
    qc.compose(eigenstate, qubits=unitary_reg, inplace=True)

    # Controlled unitary operations
    for i in range(num_ancillae):
        power = 2 ** i
        controlled_unitary = unitary.control(1)
        qc.compose(controlled_unitary, qubits=[ancilla_reg[i]] + list(unitary_reg), inplace=True, front=True)

    # Add inverse QFT to ancilla qubits
    qft_inverse = qft_dagger(num_ancillae)
    qc.append(qft_inverse, ancilla_reg)

    # Measure the ancilla qubits
    qc.measure(ancilla_reg, classical_reg)

    return qc


In [9]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute
from qiskit_aer import aer_simulator
import numpy as np
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram

# Function to create the inverse QFT
def qft_dagger(num_qubits):
    qc = QuantumCircuit(num_qubits)
    for i in range(num_qubits // 2):
        qc.swap(i, num_qubits - i - 1)
    for i in range(num_qubits):
        for j in range(i):
            qc.cp(-np.pi / (2 ** (i - j)), j, i)
        qc.h(i)
    return qc

# Function to create the QPE circuit
def phase_estimation(unitary, eigenstate, num_ancillae):
    num_unitary_qubits = unitary.num_qubits
    ancilla_reg = QuantumRegister(num_ancillae, name="ancilla")
    unitary_reg = QuantumRegister(num_unitary_qubits, name="unitary")
    classical_reg = ClassicalRegister(num_ancillae, name="classical")
    qc = QuantumCircuit(ancilla_reg, unitary_reg, classical_reg)

    # Initialize ancilla qubits in superposition
    qc.h(ancilla_reg)

    # Add eigenstate to the circuit
    qc.compose(eigenstate, qubits=unitary_reg, inplace=True)

    # Controlled unitary operations
    for i in range(num_ancillae):
        controlled_unitary = unitary.control(1)
        qc.compose(controlled_unitary, qubits=[ancilla_reg[i]] + list(unitary_reg), inplace=True, front=True)

    # Add inverse QFT to ancilla qubits
    qft_inverse = qft_dagger(num_ancillae)
    qc.append(qft_inverse, ancilla_reg)

    # Measure the ancilla qubits
    qc.measure(ancilla_reg, classical_reg)

    return qc

# Define a unitary operator (e.g., Z gate for demonstration)
unitary = QuantumCircuit(1)
unitary.z(0)  # Z-gate as a unitary operator

# Define an eigenstate (e.g., |+> state for demonstration)
eigenstate = QuantumCircuit(1)
eigenstate.h(0)  # Create |+> state

# Number of ancilla qubits
num_ancillae = 3

# Create the QPE circuit
qpe_circuit = phase_estimation(unitary, eigenstate, num_ancillae)

# Visualize the circuit
print("Quantum Phase Estimation Circuit:")
print(qpe_circuit.draw("text"))

# Simulate the circuit
simulator = aer_simulator.get_backend("qasm_simulator")
job = execute(qpe_circuit, backend=simulator, shots=1024)
result = job.result()

# Retrieve and display results
counts = result.get_counts()
print("\nMeasurement Results (Phase Estimation):")
print(counts)

# Plot the histogram of results
plot_histogram(counts)
plt.show()

# Analyze eigenvalues
# For the Z gate, the eigenvalues are ±1 (corresponding to θ = 0 or 0.5 in the binary output).
print("\nEigenvalue Analysis:")
for outcome, freq in counts.items():
    phase = int(outcome[::-1], 2) / (2 ** num_ancillae)  # Convert binary to phase
    eigenvalue = np.exp(2j * np.pi * phase)  # Eigenvalue from the phase
    print(f"Outcome: {outcome}, Phase: {phase:.2f}, Eigenvalue: {eigenvalue}")


ImportError: cannot import name 'execute' from 'qiskit' (c:\Users\noelm\AppData\Local\Programs\Python\Python312\Lib\site-packages\qiskit\__init__.py)

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.compiler import transpile, assemble
import numpy as np
import matplotlib.pyplot as plt
from qiskit.visualization import plot_histogram
def qft_dagger(num_qubits):
    qc = QuantumCircuit(num_qubits)
    for i in range(num_qubits // 2):
        qc.swap(i, num_qubits - i - 1)
    for i in range(num_qubits):
        for j in range(i):
            qc.cp(-np.pi / (2 ** (i - j)), j, i)
        qc.h(i)
    return qc

# Function to create the QPE circuit
def phase_estimation(unitary, eigenstate, num_ancillae):
    num_unitary_qubits = unitary.num_qubits
    ancilla_reg = QuantumRegister(num_ancillae, name="ancilla")
    unitary_reg = QuantumRegister(num_unitary_qubits, name="unitary")
    classical_reg = ClassicalRegister(num_ancillae, name="classical")
    qc = QuantumCircuit(ancilla_reg, unitary_reg, classical_reg)

    # Initialize ancilla qubits in superposition
    qc.h(ancilla_reg)

    # Add eigenstate to the circuit
    qc.compose(eigenstate, qubits=unitary_reg, inplace=True)

    # Controlled unitary operations
    for i in range(num_ancillae):
        controlled_unitary = unitary.control(1)
        qc.compose(controlled_unitary, qubits=[ancilla_reg[i]] + list(unitary_reg), inplace=True, front=True)

    # Add inverse QFT to ancilla qubits
    qft_inverse = qft_dagger(num_ancillae)
    qc.append(qft_inverse, ancilla_reg)

    # Measure the ancilla qubits
    qc.measure(ancilla_reg, classical_reg)

    return qc


# Define a different unitary operator (e.g., X gate for demonstration)
unitary = QuantumCircuit(1)
unitary.x(0)  # X-gate as a unitary operator

# Define a different eigenstate (e.g., |1> state for demonstration)
eigenstate = QuantumCircuit(1)
eigenstate.x(0)  # Prepare |1> state

# Number of ancilla qubits
num_ancillae = 3

# Create the QPE circuit
qpe_circuit = phase_estimation(unitary, eigenstate, num_ancillae)

# Visualize the circuit
print("Quantum Phase Estimation Circuit:")
print(qpe_circuit.draw("text"))

# Simulate the circuit
simulator = AerSimulator()
job = simulator.run(qpe_circuit, shots=1024)
result = job.result()

# Retrieve and display results
counts = result.get_counts()
print("\nMeasurement Results (Phase Estimation):")
print(counts)

# Plot the histogram of results
plot_histogram(counts)
plt.show()

Quantum Phase Estimation Circuit:
                                                               ┌───┐»
  ancilla_0: ──────────────────────────────────────────■───────┤ H ├»
                                           ┌───┐       │       └───┘»
  ancilla_1: ──────────────────────■───────┤ H ├───────┼────────────»
                                   │       ├───┤       │            »
  ancilla_2: ───────■──────────────┼───────┤ H ├───────┼────────────»
             ┌──────┴──────┐┌──────┴──────┐└───┘┌──────┴──────┐┌───┐»
    unitary: ┤ circuit-334 ├┤ circuit-334 ├─────┤ circuit-334 ├┤ H ├»
             └─────────────┘└─────────────┘     └─────────────┘└───┘»
classical: 3/═══════════════════════════════════════════════════════»
                                                                    »
«             ┌──────────────┐┌─┐      
«  ancilla_0: ┤0             ├┤M├──────
«             │              │└╥┘┌─┐   
«  ancilla_1: ┤1 circuit-358 ├─╫─┤M├───
«             │              │ ║ └╥┘

C:\Users\noelm\AppData\Local\Temp\ipykernel_5760\1302792074.py:70: DeprecationWarning: The function ``qiskit.compiler.assembler.assemble()`` is deprecated as of qiskit 1.2. It will be removed in the 2.0 release. The `Qobj` class and related functionality are part of the deprecated `BackendV1` workflow,  and no longer necessary for `BackendV2`. If a user workflow requires `Qobj` it likely relies on deprecated functionality and should be updated to use `BackendV2`.
  qobj = assemble(transpiled_circuit, shots=1024)



Eigenvalue Analysis:
Outcome: 000, Phase: 0.00, Eigenvalue: (1+0j)
